In [ ]:
import math
import os
import pickle

import numpy as np
import tensorflow as tf
from tqdm import tqdm

from tensorflow_asr.configs.config import Config
from tensorflow_asr.datasets.asr_dataset import ASRSliceDataset
from tensorflow_asr.featurizers.speech_featurizers import TFSpeechFeaturizer
from tensorflow_asr.featurizers.text_featurizers import CharFeaturizer
from tensorflow_asr.models.transducer.contextnet import ContextNet
from tensorflow_asr.optimizers.schedules import TransformerSchedule
from tensorflow_asr.utils import app_util
from tensorflow_asr.utils import env_util
from tensorflow_asr.gradient_visualisation.plotting_utils import make_directories



In [ ]:

def get_weights(net):
    return net.layers[0].get_weights()


def obtain_direction(copy_of_the_weights):
    direction1 = []
    for w in copy_of_the_weights:
        if len(w.shape) == 3:  # check for 3D tensor  or 1d-conv cnn layer  ---- this might be 2D tensor in case of low rank/depthwise seprable tensor
            random_vector = tf.random.normal(w.shape, 0, 1, tf.float32)
            w_norm_tf = tf.norm(tf.reshape(w, (w.shape[0], -1)), axis=1, keepdims=True)[:, :, None]
            d_norm1_tf = tf.norm(tf.reshape(random_vector, (random_vector.shape[0], -1)), axis=1, keepdims=True)[:, :, None]
            random_vector = random_vector * (w_norm_tf / (d_norm1_tf + 1e-10))
            direction1.append(random_vector)
        elif len(w.shape) == 4:
            random_vector = tf.random.normal(w.shape, 0, 1, tf.float32)
            w_norm_tf = tf.norm(tf.reshape(w, (w.shape[0], -1)), axis=1, keepdims=True)[:, :, None, None]
            d_norm1_tf = tf.norm(tf.reshape(random_vector, (random_vector.shape[0], -1)), axis=1, keepdims=True)[:, :, None, None]
            random_vector = random_vector * (w_norm_tf / (d_norm1_tf + 1e-10))
            direction1.append(random_vector)
        else:
            direction1.append(tf.zeros_like(w))
    return direction1


tf.keras.backend.clear_session()
env_util.setup_environment()

DEFAULT_YAML = "/Users/vaibhavsingh/Desktop/FILRCN/contextnet/config.yml"

tf.config.optimizer.set_experimental_options({"auto_mixed_precision": False})
strategy = env_util.setup_strategy([0])

config = Config(DEFAULT_YAML)
speech_featurizer = TFSpeechFeaturizer(config.speech_config)
text_featurizer = CharFeaturizer(config.decoder_config)

tf.random.set_seed(0)
test_dataset = ASRSliceDataset(
    speech_featurizer=speech_featurizer,
    text_featurizer=text_featurizer,
    **vars(config.learning_config.test_dataset_config)
)
batch_size = config.learning_config.running_config.batch_size

test_data_loader = test_dataset.create(batch_size)

number_of_points = 9
small_range = -1.0
large_range = 1.0

xcoordinates = np.linspace(small_range, large_range, num=number_of_points)
ycoordinates = np.linspace(small_range, large_range, num=number_of_points)

xcoord_mesh, ycoord_mesh = np.meshgrid(xcoordinates, ycoordinates)
inds = np.array(range(number_of_points ** 2))
s1 = xcoord_mesh.ravel()[inds]
s2 = ycoord_mesh.ravel()[inds]
coordinate = np.c_[s1, s2]

directory_to_save = make_directories("loss_lists")
model_directory = "/content/FILRCN/contextnet/checkpoints"

In [ ]:
for filename in tqdm(sorted(os.listdir(model_directory))):
    if not filename.endswith(".h5"):
        print(filename)
        continue
    loss_file = filename.split('.')[0]
    model_name = os.path.join(model_directory, filename)
    print(model_name)
    contextnet = ContextNet(**config.model_config, vocabulary_size=text_featurizer.num_classes)
    contextnet.make(speech_featurizer.shape)
    # contextnet.summary(line_length=100)
    contextnet.load_weights(model_name, by_name=True)
    contextnet.add_featurizers(speech_featurizer, text_featurizer)

    optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=contextnet.dmodel,
            warmup_steps=config.learning_config.optimizer_config.pop("warmup_steps", 10000),
            max_lr=(0.05 / math.sqrt(contextnet.dmodel))
        ),
        **config.learning_config.optimizer_config
    )

    contextnet.compile(
        optimizer=optimizer,
        steps_per_execution=1,
        global_batch_size=1,
        blank=text_featurizer.blank
    )

    converged_weights = get_weights(contextnet)

    direction1 = obtain_direction(converged_weights)
    direction2 = obtain_direction(converged_weights)

    current_direction1 = direction1
    current_direction2 = direction2
    current_loader = test_data_loader

    loss_list = np.zeros((number_of_points, number_of_points))
    acc_list_greedy_char = np.zeros((number_of_points, number_of_points))
    acc_list_beam_char = np.zeros((number_of_points, number_of_points))
    acc_list_greedy_wer = np.zeros((number_of_points, number_of_points))
    acc_list_beam_wer = np.zeros((number_of_points, number_of_points))
    col_value = 0

    index_list = []
    for count, ind in tqdm(enumerate(inds)):
        index_list.append(count)
        coord = coordinate[count]
        changes = [d0 * coord[0] + d1 * coord[1] for (d0, d1) in zip(current_direction1, current_direction2)]
        k = np.add(changes, converged_weights)
        contextnet.layers[0].set_weights(k)

        loss = contextnet.evaluate(current_loader, batch_size=batch_size, use_multiprocessing=True, workers=8)
        results = contextnet.predict(current_loader, verbose=1, use_multiprocessing=True, workers=8)
        filepath = os.path.join(os.getcwd(), "test.tsv")
        with open(filepath, "w") as openfile:
            openfile.write("PATH\tDURATION\tGROUNDTRUTH\tGREEDY\tBEAMSEARCH\n")
            for i, pred in enumerate(results):
                groundtruth, greedy, beamsearch = [x.decode('utf-8') for x in pred]
                path, duration, _ = test_dataset.entries[i]
                openfile.write(f"{path}\t{duration}\t{groundtruth}\t{greedy}\t{beamsearch}\n")

        res = app_util.evaluate_results(filepath)
        accuracy_gcer = 1 - res['greedy_cer']
        accuracy_gwer = 1 - res['greedy_wer']
        accuracy_bwer = 1 - res['beamsearch_wer']
        accuracy_bcer = 1 - res['beamsearch_cer']

        loss_list[col_value][ind % number_of_points] = loss
        acc_list_greedy_char[col_value][ind % number_of_points] = accuracy_gcer
        acc_list_greedy_wer[col_value][ind % number_of_points] = accuracy_gwer
        acc_list_beam_wer[col_value][ind % number_of_points] = accuracy_bwer
        acc_list_beam_char[col_value][ind % number_of_points] = accuracy_bcer

        ind_compare = ind + 1
        if ind_compare % number_of_points == 0:  col_value = col_value + 1
        # delete the test file which is temporary
        os.remove(filepath)

    data = {'loss_list': [loss_list],
            'greedy_char': [acc_list_greedy_char],
            'greedy_wer': [acc_list_greedy_wer],
            'beam_wer': [acc_list_beam_wer],
            'beam_char': [acc_list_beam_char]
            }

    file_path_to_save_data = os.path.join(directory_to_save, loss_file) + ".pkl"
    with open(file_path_to_save_data, 'wb') as f:
        pickle.dump(data, f)

    f.close()


In [ ]:
number_of_points = 9
small_range = -1.0
large_range = 1.0

xcoordinates = np.linspace(small_range, large_range, num=number_of_points)
ycoordinates = np.linspace(small_range, large_range, num=number_of_points)

xcoord_mesh, ycoord_mesh = np.meshgrid(xcoordinates, ycoordinates)

loss_lists_directory = os.path.join(os.getcwd(), "loss_lists")

def create_viz(loss_list, acc_list, figure_directory, filename, title="none"):
    print(filename)
    plt.figure(figsize=(5, 5))
    if title != "none":
        plt.title("Contour 2D")
    CS = plt.contour(xcoord_mesh, ycoord_mesh, loss_list, 20, zorder=1, cmap='terrain', linestyles='--')
    plt.clabel(CS, inline=1, fontsize=8)
    plt.xticks(fontsize=8, fontname="Courier New")
    plt.yticks(fontsize=8, fontname="Courier New")
    plt.savefig(figure_directory + "/original_contour/" + filename + "OriginalContour.png")

    plt.figure(figsize=(5, 5))
    if title != "none":
        plt.title("Contour 2D ")
    plt.contour(xcoord_mesh, ycoord_mesh, loss_list, 20, zorder=1, cmap='terrain', linestyles='--')
    CS = plt.contourf(xcoord_mesh, ycoord_mesh, loss_list, 20, zorder=1, cmap='terrain', linestyles='--')
    plt.clabel(CS, fontsize=8, inline=1, fmt='%2.1f')
    plt.xticks(fontsize=8, fontname="Courier New")
    plt.yticks(fontsize=8, fontname="Courier New")
    plt.colorbar(CS)
    plt.savefig(figure_directory + "/original_contour_color/" + filename + "OriginalContourColor.png")

    plt.figure(figsize=(5, 5))
    if title != "none":
        plt.title("Contour 2D Log Scale")
    CS = plt.contour(xcoord_mesh, ycoord_mesh, np.log(loss_list), 20, zorder=1, cmap='terrain', linestyles='--')
    plt.clabel(CS, inline=1, fontsize=8)
    plt.xticks(fontsize=8, fontname="Courier New")
    plt.yticks(fontsize=8, fontname="Courier New")
    plt.savefig(figure_directory + "/log_contour/" + filename + "LogScale.png")

    plt.figure(figsize=(5, 5))
    if title != "none":
        plt.title("Contour 2D Log Scale Color")
    plt.contour(xcoord_mesh, ycoord_mesh, np.log(loss_list), 20, zorder=1, cmap='terrain', linestyles='--')
    CS = plt.contourf(xcoord_mesh, ycoord_mesh, np.log(loss_list), 20, zorder=1, cmap='terrain', linestyles='--')
    plt.clabel(CS, fontsize=8, inline=1, fmt='%2.1f')
    plt.xticks(fontsize=8, fontname="Courier New")
    plt.yticks(fontsize=8, fontname="Courier New")
    plt.savefig(figure_directory + "/log_contour_color/" + filename + "LogScale.png")

    data = [
        go.Surface(
            x=xcoord_mesh, y=ycoord_mesh,
            z=(loss_list.max() - loss_list.min()) * (acc_list - acc_list.min()) / (acc_list.max() - acc_list.min() + 1e-8) + loss_list.min(),
            showscale=False, opacity=0.6, colorscale='Cividis',
        ),
        go.Surface(
            x=xcoord_mesh, y=ycoord_mesh, z=loss_list, colorscale='Jet', cmin=0, cmax=30000, opacity=0.9,
            contours=go.surface.Contours(z=go.surface.contours.Z(show=True, usecolormap=True, project=dict(z=True), ),
                                         )
        )
    ]

    layout = go.Layout(autosize=False,
                       scene=dict(dict(
                           xaxis=dict(range=[-1, 1],
                                      backgroundcolor="rgb(163, 221, 240)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", tick0=-1.5, dtick=0.5, title_font_family="Courier New"),
                           yaxis=dict(range=[-1, 1],
                                      backgroundcolor="rgb(91, 122, 133)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", tick0=-1.5, dtick=0.5, title_font_family="Courier New"),
                           zaxis=dict(range=[1, 30000],
                                      backgroundcolor="rgb(204, 231, 240)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", title_font_family="Courier New")),
                           camera=dict(eye=dict(x=2, y=2, z=1.5))),
                       margin=dict(l=50, r=10, b=20, t=10),
                       width=500, height=500)
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(
        font_family="Courier New")
    fig.update_xaxes(title_font_family="Courier New")
    fig.update_yaxes(title_font_family="Courier New")

    # iplot(fig)

    fig.write_image(figure_directory + "/loss_accuracy/" + filename + "Loss_Accuracy.png", scale=1)

    data = [
        go.Surface(
            x=xcoord_mesh, y=ycoord_mesh,
            z=(np.log(loss_list).max() - np.log(loss_list).min()) * (acc_list - acc_list.min()) / (acc_list.max() - acc_list.min() + 1e-8) + np.log(
                loss_list).min(),
            showscale=False, opacity=0.6, colorscale='Cividis',
        ),
        go.Surface(
            x=xcoord_mesh, y=ycoord_mesh, z=np.log(loss_list), colorscale='Jet', cmin=0, cmax=12, opacity=0.9,
            contours=go.surface.Contours(z=go.surface.contours.Z(show=True, usecolormap=True, project=dict(z=True), ),
                                         )
        )
    ]
    layout = go.Layout(autosize=False,
                       scene=dict(dict(
                           xaxis=dict(range=[-1, 1],
                                      backgroundcolor="rgb(163, 221, 240)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", tick0=-1.5, dtick=0.5, title_font_family="Courier New"),
                           yaxis=dict(range=[-1, 1],
                                      backgroundcolor="rgb(91, 122, 133)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", tick0=-1.5, dtick=0.5, title_font_family="Courier New"),
                           zaxis=dict(range=[1, 12],
                                      backgroundcolor="rgb(204, 231, 240)",
                                      gridcolor="white",
                                      showbackground=True,
                                      zerolinecolor="white", tick0=2, dtick=1, title_font_family="Courier New")),
                           camera=dict(eye=dict(x=2, y=2, z=1.5))),
                       margin=dict(l=50, r=10, b=20, t=10),
                       width=500, height=500)
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(
        font_family="Courier New")
    fig.update_xaxes(title_font_family="Courier New")
    fig.update_yaxes(title_font_family="Courier New")
    # iplot(fig)
    fig.write_image(figure_directory + "/log_loss_accuracy/" + filename + "Log_Loss_Accuracy.png", scale=1)


def make_directories():
    current_working_directory_abs = os.getcwd()
    figs_directory_abs = os.path.join(current_working_directory_abs, "figs")
    try:
        os.mkdir(figs_directory_abs)
        os.mkdir(os.path.join(figs_directory_abs, "log_contour"))
        os.mkdir(os.path.join(figs_directory_abs, "loss_accuracy"))
        os.mkdir(os.path.join(figs_directory_abs, "log_contour_color"))
        os.mkdir(os.path.join(figs_directory_abs, "log_loss_accuracy"))
        os.mkdir(os.path.join(figs_directory_abs, "original_contour"))
        os.mkdir(os.path.join(figs_directory_abs, "original_contour_color"))
        print(os.getcwd())
    except Exception as e:
        print("-------------Figures directory already exists-----------------")
        print("--------------The contents will be over-ridden-------------------")
        return figs_directory_abs
    return figs_directory_abs


figs_directory_abs = make_directories()


def plot_loss_landscape(figs_directory_abs):
    for file in tqdm(sorted(os.listdir(loss_lists_directory))):
        filename = os.path.join(loss_lists_directory, file)
        print("filenmes ", filename)
        with open(filename, "rb") as f:
            x_temp = pickle.load(f)
        file = file.split('.')[0]
        loss_list = x_temp['loss_list'][0]
        acc_list_greedy_char = x_temp['greedy_char'][0]
        acc_list_greedy_wer = x_temp['greedy_wer'][0]
        acc_list_beam_wer = x_temp['beam_wer'][0]
        acc_list_beam_char = x_temp['beam_char'][0]
        acc_list = acc_list_beam_char

        create_viz(loss_list, acc_list_beam_char, figs_directory_abs, file)


In [ ]:
plot_loss_landscape(figs_directory_abs)

In [ ]:
import glob
import os

import cv2

def make_directory():
    current_working_directory_abs = os.getcwd()
    video_directory_abs = os.path.join(current_working_directory_abs, "video")
    try:
        os.mkdir(video_directory_abs)
    except Exception as e:
        print("--------------video directory already exists-----------------")
        print("--------------The contents will be over-ridden-------------------")
        return video_directory_abs

    return video_directory_abs

img_array = []
size = (10,10)
figures_working_dir = os.path.join(os.getcwd(), "figs")

fname1 = figures_working_dir+'/log_loss_accuracy/*.png'
fname2 = figures_working_dir+'/log_contour/*.png'

video_directory = make_directory()


for filename1, filename2 in zip(sorted(glob.glob(fname2)), sorted(glob.glob(fname1))):
    print(filename1)
    print(filename2)

    image1 = cv2.imread(filename1)
    image2 = cv2.imread(filename2)
    height, width, layers = image1.shape
    size = (width, height)
    print(size)
    height, width, layers = image2.shape
    size = (width, height)
    print(size)

    vis = cv2.hconcat([image1, image2])
    height, width, layers = vis.shape
    size = (width, height)

    img_array.append(vis)

filename = video_directory + "/contour_video.avi"
print(filename)
out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'DIVX'), 1, size)

for i in range(len(img_array)):
    out.write(img_array[i])

out.release()
